Anaconda install instructions as of mgwr 1.0.2 and Conda 4.5:

```sh
#conda update conda
#conda update anaconda
#conda update scipy
#conda install pysal 
#pip install libpysal
#conda install mgwr
#conda install geopandas
```

In [1]:
import ee
import geemap

In [2]:
from platform import python_version

print(python_version())

3.8.13


In [3]:
import time
import multiprocessing as mp
import numpy as np
import libpysal as ps 
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap, truncate_colormap
import geopandas as gp
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from sklearn import preprocessing


In [8]:
#Load citarum dataset and generate plot of citarum counties (figure 1)
citarum_data = pd.read_csv('desa_V1.csv')
print(citarum_data)
citarum_data = citarum_data[(citarum_data['wy']>0)]
citarum_data = citarum_data.dropna()
print(citarum_data)




      FID_1          aet          dtr          eto      frac      lulc  \
0         0   692.132964   700.000000   958.160000  0.243163  1.240000   
1         2   887.393250  1500.000000  1524.000000  0.387700  3.000000   
2         3   902.069400  1383.653846  1701.352564  0.412158  8.246795   
3         4   842.699793  1367.452830  1805.386792  0.366225  8.374214   
4         5   877.632680  1500.000000  1694.214286  0.392621  8.264286   
...     ...          ...          ...          ...       ...       ...   
1204   1285   670.856243   500.000000  1888.187500  0.461129  6.000000   
1205   1286   699.274170   500.000000  1886.216216  0.470702  7.081081   
1206   1287   690.076712   500.000000  1866.236842  0.449414  8.131579   
1207   1288   645.615935   500.000000  1837.322581  0.402647  8.064516   
1208   1289  1793.268375   947.013783  1993.310873  0.837216  7.454824   

          pawc       precip           wy        POINT_X       POINT_Y  POINT_Z  
0     0.200000  2280.953203  2

In [9]:
# copy the data
df_max_scaled = citarum_data.copy()
  
# apply normalization techniques
for column in df_max_scaled.columns:
    df_max_scaled[column] = df_max_scaled[column]  / df_max_scaled[column].abs().max()
      
# view normalized data
display(df_max_scaled)

,FID_1,aet,dtr,eto,frac,lulc,pawc,precip,wy,POINT_X,POINT_Y,POINT_Z
0,0.000000,0.333241,0.466667,0.476297,0.290443,0.103333,0.952381,0.697703,0.651194,0.873928,0.990787,NaN
1,0.001552,0.427253,1.000000,0.757574,0.463082,0.250000,0.523810,0.470053,0.420988,0.872272,0.991632,NaN
2,0.002327,0.434319,0.922436,0.845735,0.492296,0.687233,0.634615,0.480004,0.387785,0.879953,0.991714,NaN
3,0.003103,0.405734,0.911635,0.897450,0.437432,0.697851,0.632225,0.527739,0.439670,0.881943,0.992161,NaN
4,0.003879,0.422553,1.000000,0.842186,0.468961,0.688690,0.523810,0.469842,0.407858,0.876675,0.991952,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1204,0.996897,0.322997,0.333333,0.938610,0.550789,0.500000,1.000000,0.396188,0.235495,0.950872,0.988861,NaN
1205,0.997673,0.336679,0.333333,0.937630,0.562223,0.590090,1.000000,0.383896,0.236038,0.949511,0.988931,NaN
1206,0.998448,0.332251,0.333333,0.927698,0.536796,0.677632,1.000000,0.377326,0.254413,0.948970,0.989135,NaN
1207,0.999224,0.310844,0.333333,0.913325,0.480935,0.672043,1.000000,0.373999,0.288467,0.947484,0.989257,NaN


In [10]:
#Prepare citarum dataset inputs
g_y = df_max_scaled['wy'].values.reshape((-1,1))
g_X = df_max_scaled[['aet','eto','frac', 'precip']].values
u = citarum_data['POINT_X']
v = citarum_data['POINT_Y']
g_coords = list(zip(u,v))

g_X = (g_X - g_X.mean(axis=0)) / g_X.std(axis=0)

g_y = g_y.reshape((-1,1))

g_y = (g_y - g_y.mean(axis=0)) / g_y.std(axis=0)

In [11]:
#Parrallelization is more favored when you your data are large and/or your machine have many many cores.
#mgwr has soft dependency of numba, please install numba if you need better performance (pip install numba).

n_proc = 2 #two processors
pool = mp.Pool(n_proc)

In [12]:
#Calibrate MGWR model (parallel processing)

start = time.time()

mgwr_selector = Sel_BW(g_coords, g_y, g_X, multi=True)
mgwr_bw = mgwr_selector.search(pool=pool)
print(mgwr_bw)
mgwr_results = MGWR(g_coords, g_y, g_X, mgwr_selector).fit(pool=pool)

end = time.time()
print('Duration (minutes): ', (end-start)/60)

Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

[44. 44. 44. 44. 44.]


Inference:   0%|          | 0/2 [00:00<?, ?it/s]

Duration (minutes):  15.379321817557017


In [13]:
mgwr_results.summary()

Model type                                                         Gaussian
Number of observations:                                                1209
Number of covariates:                                                     5

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                             92.923
Log-likelihood:                                                    -164.484
AIC:                                                                338.969
AICc:                                                               341.038
BIC:                                                              -8452.526
R2:                                                                   0.923
Adj. R2:                                                              0.923

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

In [14]:
dfMGWR = pd.DataFrame(mgwr_results.params)
dfMGWR.to_csv('dfMGWR_5param.csv', encoding='utf-8')

In [15]:
# Calibrate GWR model (parallel processing)

start = time.time()

gwr_selector = Sel_BW(g_coords, g_y, g_X)
gwr_bw = gwr_selector.search(pool=pool)
print(gwr_bw)
gwr_results = GWR(g_coords, g_y, g_X, gwr_bw).fit(pool=pool)

end = time.time()
print('Duration (minutes): ', (end-start)/60)

52.0
Duration (minutes):  0.06202141046524048


In [ ]:
dfGWR = pd.DataFrame(gwr_results.params)
dfGWR.to_csv('dfGWR_5param.csv', encoding='utf-8')

In [ ]:
citarum_shp = gp.read_file('das_citarum_desa_Data_48S_S1.shp')
fig, ax = plt.subplots(figsize=(10,10))
citarum_shp.plot(ax=ax, **{'edgecolor':'black', 'facecolor':'white'})
citarum_shp.centroid.plot(ax=ax, c='black')

In [ ]:
#Prepare GWR results for mapping

#Add GWR parameters to GeoDataframe
citarum_shp['gwr_intercept'] = gwr_results.params[:,0]*100

display(citarum_shp['gwr_intercept'])

#Obtain t-vals filtered based on multiple testing correction
gwr_filtered_t = gwr_results.filter_tvals(alpha=0.05)
display(gwr_filtered_t)

In [ ]:
#Prepare MGWR results for mapping

#Add MGWR parameters to GeoDataframe
citarum_shp['mgwr_intercept'] = mgwr_results.params[:,0]*100

display(citarum_shp['mgwr_intercept'])

#Obtain t-vals filtered based on multiple testing correction
mgwr_filtered_t = mgwr_results.filter_tvals(alpha=0.05)
display(mgwr_filtered_t)

In [ ]:
#Comparison maps of GWR vs. MGWR parameter surfaces where the grey units pertain to statistically insignificant parameters

#Prep plot and add axes
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(45,20))
ax0 = axes[0]
ax0.set_title('GWR Intercept Surface (BW: ' + str(gwr_bw) +')', fontsize=40)
ax1 = axes[1]
ax1.set_title('MGWR Intercept Surface (BW: ' + str(mgwr_bw[0]) +')', fontsize=40)

#Set color map
cmap = plt.cm.seismic

#Find min and max values of the two combined datasets
gwr_min = citarum_shp['gwr_intercept'].min()
gwr_max = citarum_shp['gwr_intercept'].max()
mgwr_min = citarum_shp['mgwr_intercept'].min()
mgwr_max = citarum_shp['mgwr_intercept'].max()
vmin = -300
vmax = 300


cmap = truncate_colormap(cmap, vmin,vmax)


#Create scalar mappable for colorbar and stretch colormap across range of data values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

#Plot GWR parameters
citarum_shp.plot('gwr_intercept', cmap=sm.cmap, ax=ax0, vmin=vmin, vmax=vmax, **{'edgecolor':'black', 'alpha':.65})
#If there are insignificnt parameters plot gray polygons over them
if (gwr_filtered_t[:,0] == 0).any():
    citarum_shp[gwr_filtered_t[:,0] == 0].plot(color='lightgrey', ax=ax0, **{'edgecolor':'black'})

#Plot MGWR parameters
citarum_shp.plot('mgwr_intercept', cmap=sm.cmap, ax=ax1, vmin=vmin, vmax=vmax, **{'edgecolor':'black', 'alpha':.65})
#If there are insignificnt parameters plot gray polygons over them
if (mgwr_filtered_t[:,0] == 0).any():
    citarum_shp[mgwr_filtered_t[:,0] == 0].plot(color='lightgrey', ax=ax1, **{'edgecolor':'black'})
 
#Set figure options and plot 
fig.tight_layout()    
fig.subplots_adjust(right=0.9)
cax = fig.add_axes([0.92, 0.14, 0.03, 0.75])
sm._A = []
cbar = fig.colorbar(sm, cax=cax)
cbar.ax.tick_params(labelsize=50) 
ax0.get_xaxis().set_visible(False)
ax0.get_yaxis().set_visible(False)
ax1.get_xaxis().set_visible(False)
ax1.get_yaxis().set_visible(False)
plt.show()

In [ ]:
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_image
from functools import partial
from rasterio.enums import MergeAlg

In [ ]:
    # Using GeoCube to rasterize the Vector
    citarum_shp_GEO = citarum_shp.to_crs('EPSG:4326')

    shptoraster1 = make_geocube(
        vector_data=citarum_shp_GEO,
        measurements=["mgwr_intercept"],
        resolution=(-0.001, 0.001),
        fill = 0
    )
    
    # Save raster census raster
    shptoraster1.rio.to_raster('mgwr1_geo.tiff')

In [ ]:
import ee
import geemap


In [ ]:
Map = geemap.Map(center=(-6.3893832360229545,107.31268018020506), zoom=7)
Map


In [ ]:
citarum_shp.to_file('tes2.shp')  

In [ ]:
test1 = 'tes2.shp'
citarum_kecamatan = geemap.shp_to_ee(test1)

Map.addLayer(citarum_kecamatan, {}, 'citarum_kecamatan')

In [ ]:
palette = ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
Map.add_styled_vector(citarum_kecamatan, column="mgwr_inter", palette=palette, layer_name="Styled vector")


In [ ]:
palette = ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
Map.add_styled_vector(citarum_kecamatan, column="mgwr_inter", palette=palette, layer_name="Styled vector")


In [ ]:
text = 'Hello World'
Map.add_text(text, position='bottomright')

In [ ]:
html = """
<h2>Jupyter Logo</h2>
<img src="https://upload.wikimedia.org/wikipedia/commons/d/dd/Alt_Logo_BRIN.png">
"""
Map.add_html(html, position='bottomright')

In [ ]:
palette1=  ['00ff00', 'ff0000', '006622']
Map.add_styled_vector(citarum_kecamatan, column="mgwr_inter", palette=palette1, layer_name="Tes1")



In [ ]:
mgwr2 = 'mgwr1_geo.tiff'
ndviParams = {'palette': ['#d73027', '#f46d43', '#fdae61',
                          '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850'],
              'min': -100,
              'max': 3000}

Map.add_raster(mgwr2, ndviParams, layer_name='mgwr12435')

In [ ]:
mgwr1 = 'mgwr1_geo.tiff'
Map.add_raster(mgwr1, {}, layer_name='mgwr11')


In [ ]:
import os
print(os.getcwd() + "\n")



In [57]:
tes1 = os.getcwd() + '\LULC.tif'
print(tes1 + "\n")

Map.add_raster(tes1, {}, layer_name='LULC')


E:\citarum\jupyter\LULC.tif



In [53]:
Map.clear()

Map(bottom=68166.20001220703, center=[-6.3868173785243965, 107.63056957773871], crs={'name': 'EPSG3857', 'cust…

In [54]:
Map = geemap.Map(center=(-6.3893832360229545,107.31268018020506), zoom=7)
Map

Map(center=[-6.3893832360229545, 107.31268018020506], controls=(WidgetControl(options=['position', 'transparen…

In [56]:
url = 'https://github.com/giswqs/data/raw/main/raster/srtm90.tif'
filename = 'srtm90.tif'
geemap.download_file(url, filename)

Map = geemap.Map()
Map.add_raster(filename, palette='terrain', layer_name="DEM")
vis_params = {'min': 0, 'max': 4000, 'palette': 'terrain'}
Map.add_colorbar(vis_params, label='Elevation (m)')
Map

Downloading...
From: https://github.com/giswqs/data/raw/main/raster/srtm90.tif
To: E:\citarum\jupyter\srtm90.tif
100%|██████████████████████████████████████████████████████████████████████████████| 15.2M/15.2M [00:20<00:00, 744kB/s]


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [58]:
Map.add_raster('mgwr1_geo.tiff', palette='terrain', layer_name="mgwr1_geo.tiff")

In [66]:
mgwr2 = 'mgwr1_geo.tiff'
Map.add_raster(mgwr2,  palette='Greens',layer_name='tes122')

In [70]:
raster = rasterio.open('mgwr1_geo.tiff')

show(raster)

NameError: name 'rasterio' is not defined

In [69]:
geemap.plot_raster('mgwr1_geo.tiff', cmap='terrain', figsize=(15, 10))


ContextualVersionConflict: (xarray 2022.12.0 (c:\users\fardh\anaconda3\envs\mgwr38-geemap\lib\site-packages), Requirement.parse('xarray<2022.9.0,>2022.3.0'), {'pyvista-xarray'})

In [71]:
# Define the colors you want
cmap = ListedColormap(["white", "tan", "springgreen", "darkgreen"])

# Define a normalization from values -> colors
norm = colors.BoundaryNorm([0, 2, 10, 20, 30], 5)

NameError: name 'ListedColormap' is not defined